In [ ]:
%pip install tqdm

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
from os import path
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from models.commit import Commit
from models.file import File

In [ ]:
def process_commit(com: Commit):
    repo_path = path.join('..', 'download', 'orgs', com.org_name, com.repo_name)
    
    processed_files = set()
    
    futures = set()
    
    with ThreadPoolExecutor(max_workers=25) as executor:
        for file in com.files:
            file_key = (file, com.sha, com.repo_name, com.org_name)
            if file_key in processed_files:
                continue
            processed_files.add(file_key)
            futures.add(executor.submit(File.get_file_content, repo_path, com.sha, file))
        
        if File.exists_in_batches(processed_files):
            return
        
        files_to_db = []
        for future in as_completed(futures):
            file_content, file_name = future.result()
            file_to_db = File(file_name, com.sha, com.repo_name, com.org_name, file_name.split('.')[-1].lower(), file_content)
            files_to_db.append(file_to_db)
        
        if files_to_db:
            File.add_files_in_batches(files_to_db)


In [ ]:
commits = Commit.fetch_all_commits()
parent_folder = path.join('..', 'download', 'orgs')

In [ ]:
for com in tqdm(commits, total=len(commits), desc="Processing commits"):
    if com.files:
        process_commit(com)